In [0]:
#importing libraries
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [0]:
from google.colab import files
files.upload()

{}

In [0]:
#importing data.
my_data=pd.read_excel("default of credit card clients (1).xls")
print(my_data.index)

Index([ 'ID',     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       29991, 29992, 29993, 29994, 29995, 29996, 29997, 29998, 29999, 30000],
      dtype='object', length=30001)


In [0]:

my_data=my_data.drop(my_data.index[0],axis=0)
my_data.head()
X=my_data.drop("Y",axis=1)
Y=my_data["Y"]

In [0]:
#preprocessing data.
x_col=X.columns
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3)
scaler=StandardScaler()
x_train=pd.DataFrame(data=scaler.fit_transform(x_train),columns=x_col)
x_test=pd.DataFrame(data=scaler.transform(x_test),columns=x_col)
y_train=np.array(y_train,dtype=np.float64)
y_train=y_train.reshape(21000,1)

In [0]:
print(x_train.shape)
print(y_train.shape)

(21000, 23)
(21000, 1)


In [0]:
#Helper funtions and computational graph.
def init_weights(shape):
    return tf.Variable(initial_value=tf.random_normal(shape,stddev=0.01))
                       
def my_model(X,wh,wh2,w_out,keep_input,keep_hidden):
    X=tf.nn.dropout(X,keep_input)
    h1=tf.nn.relu(tf.matmul(X,wh))
    h1=tf.nn.dropout(h1,keep_hidden)
    h2=tf.nn.relu(tf.matmul(h1,wh2))
    h2=tf.nn.dropout(h2,keep_hidden)
    output_layer=tf.matmul(h2,w_out)
    return output_layer
                 


In [0]:
#Defining placeholders and optimizers.
X=tf.placeholder("float",[None,23])
Y=tf.placeholder("float",[None,1])

wh=init_weights([23,625])
wh2=init_weights([625,625])
w_out=init_weights([625,1])

keep_input=tf.placeholder("float")
keep_hidden=tf.placeholder("float")

p_y=my_model(X,wh,wh2,w_out,keep_input,keep_hidden)

cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=p_y,labels=Y))
optimizer=tf.train.AdamOptimizer(0.01)
train=optimizer.minimize(cost)
predictions=tf.sigmoid(p_y)
saver=tf.train.Saver()








In [0]:
#initializing variables.
init=tf.global_variables_initializer()
num_iter=1000

In [0]:
#Training model by feeding in batches of data in session.Saving the model at end.
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_iter):
        for start,end in zip(range(0,len(x_train),128),range(128,len(x_train)+1,128)):
            sess.run(train,feed_dict={X:x_train[start:end],Y:y_train[start:end],keep_input:0.8,keep_hidden:0.5})
        print(i,sess.run(cost,feed_dict={X:x_train,Y:y_train,keep_input:1,keep_hidden:1}))
        #print(i,np.mean(y_train==sess.run(predictions>0.5,feed_dict={X:x_train,Y:y_train,keep_input:1.0,keep_hidden:1.0})))
    saver.save(sess,"TF_imp_2/tf_imp_2.ckpt")
    
    

0 0.4913569
1 0.47293252
2 0.47585198
3 0.45587352
4 0.47155157
5 0.46066937
6 0.46455428
7 0.46374828
8 0.45465517
9 0.4627201
10 0.465535
11 0.46232772
12 0.45882097
13 0.491472
14 0.47981334
15 0.47851348
16 0.48462743
17 0.46163422
18 0.45876017
19 0.4706744
20 0.46915942
21 0.4723172
22 0.4561981
23 0.46567234
24 0.4634682
25 0.4609317
26 0.4554644
27 0.47365943
28 0.4640875
29 0.47035632
30 0.47033483
31 0.4671371
32 0.47183004
33 0.45672294
34 0.46745968
35 0.46172172
36 0.45318684
37 0.46839824
38 0.45533863
39 0.46400136
40 0.46888104
41 0.47471532
42 0.46356696
43 0.4587553
44 0.4671352
45 0.45556158
46 0.45968777
47 0.47850668
48 0.45957923
49 0.4578219
50 0.4718099
51 0.45680526
52 0.4756874
53 0.48052362
54 0.46264166
55 0.47694084
56 0.45757207
57 0.46719655
58 0.47093
59 0.46855742
60 0.45539173
61 0.45702168
62 0.4806323
63 0.46575224
64 0.48859087
65 0.45362058
66 0.47396186
67 0.45729157
68 0.4592972
69 0.4629093
70 0.46721908
71 0.4615452
72 0.46974227
73 0.455767
74

In [0]:
#Restoring the previously saved session.
with tf.Session() as sess:
  saver.restore(sess,"TF_imp_2/tf_imp_2.ckpt")
  pred_values=sess.run(predictions,feed_dict={X:x_train,keep_input:1,keep_hidden:1})

INFO:tensorflow:Restoring parameters from TF_imp_2/tf_imp_2.ckpt


In [0]:
#printing the predicted values by the model.
pred_values

array([[0.06067451],
       [0.5       ],
       [0.07091356],
       ...,
       [0.5       ],
       [0.3850063 ],
       [0.16643453]], dtype=float32)

In [0]:
#To check how many of the predicted probablities are 1!
count=0
for i in pred_values:
  if i>0.5:
    count+=1

In [0]:
#show count
count

2960

In [0]:
#initialize count2 varaible
count2=0

In [0]:
#To check how many actual true labels were in training_data
for j in y_train:
  if j==1:
    count2+=1
    

In [0]:
#Number of actual true labels in training_data.
count2

4645

In [0]:
#Masking the y_pred values into 0's and 1's
y_pred=pred_values>0.5

In [0]:
#Final accuracy of the model.
np.mean(y_pred==y_train)

0.8242380952380952